In [ ]:
from tensorflow.keras import regularizers
import tensorflow as tf
import os
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
training_dir="./data/train"
validation_dir="./data/test"

In [ ]:
training_datagen=ImageDataGenerator(
    rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# training_datagen=ImageDataGenerator(
#     rescale=1./255)
     
train_generator=training_datagen.flow_from_directory(
training_dir,
class_mode='categorical',
target_size=(150,150))


validation_datagen=ImageDataGenerator(rescale=1./255)

valid_generator=validation_datagen.flow_from_directory(
validation_dir,
class_mode='categorical',
target_size=(150,150))

In [ ]:
# ------------------------------------------case 1.) ----------------------------------------

model =tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation ='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
# ----------------------------------- case 2.)-------------------------------------
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)

prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)


inputs = tf.keras.Input(shape=(160, 160, 3))

x = base_model(inputs, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
checkpoint_path="./checkpoint/cp.ckpt"
checkpoint_dir=os.path.dirname(checkpoint_path)

cp_callback=tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1)

history=model.fit_generator(train_generator,
                            steps_per_epoch=10,
                            epochs=60,
                            validation_data =valid_generator,
                            verbose=2,
                           callbacks=[cp_callback])   

In [ ]:
model.save('model/breed_final')